In [18]:
from datetime import datetime, timedelta
import json
import requests
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def create_test_incident():
    now = datetime.now()
    first_event_time = now - timedelta(minutes=5)
    
    test_incident = {
        "id": f"TEST_{int(now.timestamp())}",
        "title": "Blue Iris Alert",  # Added required title
        "severity": "high",          # Match enum string
        "source": "wazuh",
        "description": json.dumps({
            "rule_id": "5710",
            "rule_description": "Blue Iris Alert",
            "rule_level": 10,
            "total_events": 2,
            "agent_names": ["Blue Iris"],
            "first_event_timestamp": first_event_time.isoformat(),
            "last_event_timestamp": now.isoformat(),
            "sample_event": {
                "rule": {
                    "id": "5710",
                    "description": "Blue Iris Alert",
                    "level": 10
                },
                "agent": {
                    "name": "Blue Iris",
                    "id": "001"
                },
                "timestamp": first_event_time.isoformat(),
            },
            "csv_path": "/incidents/test/events.csv"
        }),
        "acknowledged": False,
        "escalated": False,
        "created_at": now.isoformat()  # Match datetime format
    }

    try:
        response = requests.post(
            'https://0.0.0.0:8334/incidents/',
            json=test_incident,
                        headers={
                'Content-Type': 'application/json',
                'X-API-Key': "davis"},
            verify=False
        )
        response.raise_for_status()
        logger.info(f"Status Code: {response.status_code}")
        logger.info(f"Response: {response.json()}")
        
    except requests.exceptions.RequestException as e:
        logger.error(f"Error sending incident: {e}")
        if hasattr(e.response, 'text'):
            logger.error(f"Response body: {e.response.text}")

if __name__ == "__main__":
    create_test_incident()

/Users/fabian/anaconda3/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host '0.0.0.0'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
INFO:__main__:Status Code: 200
INFO:__main__:Response: {'message': 'Incident created', 'id': 'TEST_1736454618'}


In [15]:
from sqlalchemy import create_engine, Column, Integer, String, DateTime
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from datetime import datetime
import bcrypt
import json
import os

SQLALCHEMY_DATABASE_URL = "sqlite:///./users.db"
engine = create_engine(SQLALCHEMY_DATABASE_URL, connect_args={"check_same_thread": False})
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base()

class User(Base):
    __tablename__ = "users"

    id = Column(Integer, primary_key=True)
    username = Column(String(100), unique=True)  # Will use email as username
    name = Column(String(100))
    email = Column(String(100), unique=True)
    phone = Column(String(20))
    department = Column(String(50))
    role = Column(String(20))
    password_hash = Column(String(100))
    created_at = Column(DateTime, default=datetime.utcnow)
    updated_at = Column(DateTime, default=datetime.utcnow)

def hash_password(password):
    return bcrypt.hashpw(password.encode(), bcrypt.gensalt()).decode()

def create_initial_users():
    try:
        # Drop and recreate tables
        Base.metadata.drop_all(bind=engine)
        Base.metadata.create_all(bind=engine)
        
        db = SessionLocal()

        # Initial users with email as username
        users = [
            User(
                username="fabianannomerz@gmail.com",  # Email as username
                name="Fabian Merz",
                email="fabianannomerz@gmail.com",
                phone="**621",
                department="Security",
                role="admin",
                password_hash=hash_password("admin")
            ),
            User(
                username="merzfabi@hs-albsig.de",  # Email as username
                name="Fabian Merz Student", 
                email="merzfabi@hs-albsig.de",
                phone="**622",
                department="IT",
                role="analyst",
                password_hash=hash_password("analyst")
            )
        ]

        for user in users:
            db.add(user)
            print(f"Creating user: {user.name} (username: {user.username})")

        db.commit()
        print(f"\nSuccessfully created {len(users)} users")

    except Exception as e:
        print(f"Error: {str(e)}")
        db.rollback()
        raise
    finally:
        db.close()

if __name__ == "__main__":
    create_initial_users()

Creating user: Fabian Merz (username: fabianannomerz@gmail.com)
Creating user: Fabian Merz Student (username: merzfabi@hs-albsig.de)

Successfully created 2 users


In [28]:
def print_database():
    try:
        db = SessionLocal()
        users = db.query(User).all()
        
        print("\n=== Database Contents ===")
        print("-" * 150)
        print(f"{'ID':<4} {'Username':<30} {'Name':<20} {'Role':<10} {'Department':<15} {'Password Hash':<50}")
        print("-" * 150)
        
        for user in users:
            print(f"{user.id:<4} {user.username:<30} {user.name:<20} {user.role:<10} "
                  f"{user.department:<15} {user.password_hash:<50}")
        
        print("-" * 150)
        print(f"Total users: {len(users)}")
        
    except Exception as e:
        print(f"Error querying database: {str(e)}")
    finally:
        db.close()

if __name__ == "__main__":
    print_database()


=== Database Contents ===
------------------------------------------------------------------------------------------------------------------------------------------------------
ID   Username                       Name                 Role       Department      Password Hash                                     
------------------------------------------------------------------------------------------------------------------------------------------------------
1    fabianannomerz@gmail.com       Fabian Merz          admin      Security        $2b$12$5andMgaVHwcRCiuNAd9k4Oy26cKRN4YU7y/v/zjvmnxpPnVIj0SAK
2    merzfabi@hs-albsig.de          Fabian Merz Student  analyst    IT              $2b$12$EsDssfS0myULH7j1QSJsZurmMFqXkF26Ytdz3Y5J80qBa6iAzqqay
------------------------------------------------------------------------------------------------------------------------------------------------------
Total users: 2
